# 作业

请画出下方图片中的6号、3号和5号节点的从1层到3层到WL子树。
![tree1](tree0.png)

## 节点3
![tree1](tree1.png)
## 节点5
![tree1](tree2.png)
## 节点6
![tree1](tree3.png)

# 笔记
## 基于图同构网络(GIN)的图表征网络的实现
基于图同构网络的图表征学习主要包含以下两个过程:
1. 首先计算得到节点表征;
2. 其次对图上各个节点的表征做图池化(Graph Pooling)，或称为图读出
(Graph Readout)，得到图的表征(Graph Representation)。
在此文中，我们将采用自顶向下的方式，来学习基于图同构模型(GIN)的图表征 学习方法。我们首先关注如何基于节点表征计算得到图的表征，而忽略计算结点表 征的方法。
 
## 基于图同构网络的图表征模块
 此模块首先采用GINNodeEmbedding模块对图上每一个节点做节点嵌入(Node Embedding)，得到节点表征;然后对节点表征做图池化得到图的表征;最后用一 层线性变换对图表征转换为对图的预测。
 
 接下来我们将学习节点嵌入的方法。
## 基于图同构网络的节点嵌入模块
 此节点嵌入模块基于多层GINConv实现结点嵌入的计算。此处我们先忽略GINConv的 实现。输入到此节点嵌入模块的节点属性为类别型向量，我们首先用AtomEncoder 对其做嵌入得到第0层节点表征(稍后我们再对AtomEncoder做分析)。然后我们逐 层计算节点表征，从第1层开始到第num_layers层，每一层节点表征的计算都以上 一层的节点表征h_list[layer]、边edge_index和边的属性edge_attr为输入。需要 注意的是，GINConv的层数越多，此节点嵌入模块的感受野(receptive field)越 大，结点i的表征最远能捕获到结点i的距离为num_layers的邻接节点的信息。
 
 接下来我们来学习图同构网络的关键组件GINConv。
 
 图同构卷积层的数学定义如下:
 
$$
\mathbf{x}^{\prime}_i = h_{\mathbf{\Theta}} \left( (1 + \epsilon) \cdot
\mathbf{x}_i + \sum_{j \in \mathcal{N}(i)} \mathbf{x}_j \right)
$$

PyG中已经实现了此模块，我们可以通过torch_geometric.nn.GINConv来使用PyG 定义好的图同构卷积层，然而该实现不支持存在边属性的图。在这里我们自己自定 义一个支持边属性的GINConv模块。

 
 由于输入的边属性为类别型，因此我们需要先将类别型边属性转换为边表征。我们 定义的GINConv模块遵循“消息传递、消息聚合、消息更新”这一过程。
这一过程随着self.propagate()方法的调用开始执行，该函数接收edge_index, x, edge_attr此 三 个 函 数 。edge_index是 形 状 为[2,num_edges]的 张 量 (tensor)。 在消息传递过程中，此张量首先被按行拆分为x_i和x_j张量，x_j表示了消息传 递的源节点，x_i表示了消息传递的目标节点。 接着message()方法被调用，此函数定义了从源节点传入到目标节点的消息，在 这里要传递的消息是源节点表征与边表征之和的relu()的输出。我们在 super(GINConv, self).__init__(aggr = "add")中定义了消息聚合方式为add， 那么传入给任一个目标节点的所有消息被求和得到aggr_out，它还是目标节点的 中间过程的信息。
接着执行消息更新过程，我们的类GINConv继承了MessagePassing类，因此 update()函数被调用。然而我们希望对节点做消息更新中加入目标节点自身的消 息，因此在update函数中我们只简单返回输入的aggr_out。
然 后 在forward函 数 中 我 们 执 行out = self.mlp((1 + self.eps) *x + self.propagate(edge_index, x=x, edge_attr=edge_embedding))实现消息的更 新。